In [1]:
import torch
import time
import sys
sys.path.append("/append/MINDiff/path")
from pipeline_stable_diffusion_mask import StableDiffusionMaskPipeline
from set_attn_proc import set_mask_attn
from mask_attention_processor import MaskAttnProcessor2_0

/home/seulgi/anaconda3/envs/dreambooth/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pipeline = StableDiffusionMaskPipeline.from_pretrained(
    "/Input/model/path",
    torch_dtype=torch.float16).to("cuda")
unet = pipeline.unet
unet = set_mask_attn(
    unet=unet,
    mask_resolution=16,
    )
mask_token_id = pipeline.tokenizer.convert_tokens_to_ids("sks</w>")

Loading pipeline components...: 100%|██████████████████████████████████████| 7/7 [00:26<00:00,  3.84s/it]


In [3]:
def measure_sd_latency(pipe, prompt, pre_prompt, num_images=1, repeat=10):
    # Warm-up
    MaskAttnProcessor2_0.mask = None
    _ = pipe(prompt,
             num_images_per_prompt=num_images,
             mask_token_id=mask_token_id,
             pre_prompt=pre_prompt,
             attn_scale=1.0)
    if hasattr(MaskAttnProcessor2_0, 'mask_buffer'):
        del MaskAttnProcessor2_0.mask_buffer
        torch.cuda.empty_cache()
    torch.cuda.synchronize() # Wait until GPU operations are complete

    # Wait until GPU operations are complete
    torch.cuda.reset_peak_memory_stats() # Reset peak memory usage from previous runs
    
    start = time.time()
    for _ in range(repeat):
        MaskAttnProcessor2_0.mask = None
        _ = pipe(prompt,
                 num_images_per_prompt=num_images,
                 mask_token_id=mask_token_id,
                 attn_scale=1.0,
                 pre_prompt=pre_prompt)
        if hasattr(MaskAttnProcessor2_0, 'mask_buffer'):
            del MaskAttnProcessor2_0.mask_buffer
            torch.cuda.empty_cache()
    torch.cuda.synchronize()
    end = time.time()
    
    avg_latency = (end - start) / repeat
    peak_memory = torch.cuda.max_memory_allocated() / 1024**2
    
    return avg_latency, peak_memory

In [4]:
latency, memory = measure_sd_latency(pipeline, "a sks backpack on a wooden table", "a sks backpack", num_images=1, repeat=10)
print(f"Avg latency: {latency*1000:.2f} ms")
print(f"Peak GPU memory: {memory:.2f} MB")

100%|████████████████████████████████████████████████████████████████████| 50/50 [00:02<00:00, 23.74it/s]


Avg latency: 2213.99 ms
Peak GPU memory: 3269.29 MB


In [ ]:
def measure_sd_latency_event(pipe, prompt, pre_prompt, num_images=1, repeat=10):
    MaskAttnProcessor2_0.mask = None
    _ = pipe(prompt,
             num_images_per_prompt=num_images,
             mask_token_id=mask_token_id,
             pre_prompt=pre_prompt,
             attn_scale=1.0) # wram-up
    torch.cuda.synchronize()
    torch.cuda.reset_peak_memory_stats()

    total_time = 0.0

    for _ in range(repeat):
        start_event = torch.cuda.Event(enable_timing=True)
        end_event = torch.cuda.Event(enable_timing=True)

        start_event.record()
        MaskAttnProcessor2_0.mask = None
        _ = pipe(prompt,
                 num_images_per_prompt=num_images,
                 mask_token_id=mask_token_id,
                 attn_scale=1.0,
                 pre_prompt=pre_prompt)
        end_event.record()

        torch.cuda.synchronize()
        elapsed = start_event.elapsed_time(end_event)
        total_time += elapsed

    avg_latency = total_time / repeat
    peak_memory = torch.cuda.max_memory_allocated() / 1024**2

    return avg_latency, peak_memory

In [10]:
latency, memory = measure_sd_latency_event(pipeline, "a sks backpack on a wooden table", "a sks backpack", num_images=1, repeat=30)
print(f"Avg latency: {latency:.2f} ms") # No need to multiply by 1000 since the result is already in milliseconds
print(f"Peak GPU memory: {memory:.2f} MB")

100%|████████████████████████████████████████████████████████████████████| 50/50 [00:02<00:00, 23.90it/s]

Avg latency: 2174.57 ms
Peak GPU memory: 3265.62 MB


In [6]:
batch_sizes = [1, 2, 4, 8]
prompt = "a sks backpack on a wooden table"

for batch_size in batch_sizes:

    print(f"\nBatch size: {batch_size}")

    # Warm-up run
    MaskAttnProcessor2_0.mask = None
    MaskAttnProcessor2_0.stack = None
    _ = pipeline(prompt,
                num_images_per_prompt=batch_size,
                mask_token_id=mask_token_id,
                attn_scale=1.0,
                pre_prompt="a sks backpack")
    if hasattr(MaskAttnProcessor2_0, 'mask_buffer'):
            del MaskAttnProcessor2_0.mask_buffer
            torch.cuda.empty_cache()
    
    # Start measurement
    torch.cuda.synchronize()
    start = time.time()
    MaskAttnProcessor2_0.mask = None
    _ = pipeline(prompt,
                num_images_per_prompt=batch_size,
                mask_token_id=mask_token_id,
                attn_scale=1.0,
                pre_prompt="a sks backpack")
    if hasattr(MaskAttnProcessor2_0, 'mask_buffer'):
            del MaskAttnProcessor2_0.mask_buffer
            torch.cuda.empty_cache()
    torch.cuda.synchronize()
    end = time.time()

    elapsed = (end - start)
    elapsed_ms = elapsed * 1000
    fps = batch_size / elapsed
    per_image_time = elapsed_ms / batch_size

    print(f"Time: {elapsed_ms:.2f}ms | FPS: {fps:.2f} | Per image: {per_image_time:.2f}ms")


Batch size: 1


100%|████████████████████████████████████████████████████████████████████| 50/50 [00:02<00:00, 23.80it/s]


Time: 2204.41ms | FPS: 0.45 | Per image: 2204.41ms

Batch size: 2


100%|████████████████████████████████████████████████████████████████████| 50/50 [00:03<00:00, 15.40it/s]


Time: 3430.85ms | FPS: 0.58 | Per image: 1715.42ms

Batch size: 4


100%|████████████████████████████████████████████████████████████████████| 50/50 [00:06<00:00,  7.99it/s]


Time: 6598.92ms | FPS: 0.61 | Per image: 1649.73ms

Batch size: 8


100%|████████████████████████████████████████████████████████████████████| 50/50 [00:11<00:00,  4.35it/s]


Time: 12140.12ms | FPS: 0.66 | Per image: 1517.51ms
